In [1]:
from zipline.optimize.objectives import (
    TargetWeights, 
    MaximizeAlpha,
    QuadraticUtility,
)
from zipline.optimize.risks import (
    StaticCovar,
    EmpyricalCovar,
    FactorModelCovar,
)
from zipline.optimize.constraints import (
    MaxGrossExposure,
    NetExposure,
    DollarNeutral,
    NetGroupExposure,
    PositionConcentration,
    FactorExposure,
    Pair,
    Basket,
    Frozen,
    LongOnly,
    ShortOnly,
    FixedWeight,
    CannotHold,
    MaxWeights,
    MinWeights,
)
from zipline.optimize.core import calculate_optimal_portfolio

import numpy as np
import pandas as pd
import cvxpy as cvx

In [2]:
rets = np.array(
    [0.0644,0.0806,0.1205,0.0426,0.0329,0.048,0.0884,0.0665]
)
stdev = np.array(
    [0.3118,0.3359,0.3399,0.3249,0.3374,0.3149,0.3549,0.3759]
)
corr = np.array(
[[1,0.049,0.12,-0.109,-0.2,-0.023,0.116,-0.252],
[0.049,1,0.093,0.101,0.155,-0.128,-0.196,0.109],
[0.12,0.093,1,-0.086,0.04,0.108,0.248,0.023],
[-0.109,0.101,-0.086,1,0.028,-0.235,0.008,0.212],
[-0.2,0.155,0.04,0.028,1,-0.179,0.031,0.193],
[-0.023,-0.128,0.108,-0.235,-0.179,1,0.265,-0.127],
[0.116,-0.196,0.248,0.008,0.031,0.265,1,0.028],
[-0.252,0.109,0.023,0.212,0.193,-0.127,0.028,1]]
)
stocks = [str(i).zfill(6) for i in range(1, 9)]

In [11]:
cov = np.diag(stdev) @ corr @ np.diag(stdev)
cov_df = pd.DataFrame(cov, index=stocks, columns=stocks)

stdev_s = pd.Series(stdev, index=stocks)
corr_df = pd.DataFrame(corr, index=stocks, columns=stocks)

i = 6
alphas = pd.Series(rets[:i], index=stocks[:i])

risk_model = StaticCovar(cov_df)
# risk_model = StaticCovar(corr=corr_df, stdev=stdev_s)

alpha_model = MaximizeAlpha(alphas)
lambda_ = 2

obj1 = QuadraticUtility(alpha_model, risk_model, lambda_)
obj2 = QuadraticUtility(None, risk_model, lambda_)
obj3 = QuadraticUtility(alpha_model, None, lambda_)

In [12]:
con1 = NetExposure(1, 1)
con2 = MaxGrossExposure(1)

In [13]:
cond = [con1, con2]

print(calculate_optimal_portfolio(obj1, cond, None))
print(calculate_optimal_portfolio(obj2, cond, None))
print(calculate_optimal_portfolio(obj3, cond, None))

000001    0.179149
000002    0.142654
000003    0.222875
000004    0.167579
000005    0.104241
000006    0.183502
dtype: float64
000001    0.204827
000002    0.082524
000003    0.061248
000004    0.165623
000005    0.149755
000006    0.217528
000007    0.025279
000008    0.093217
dtype: float64
000001    3.779190e-10
000002    5.369911e-10
000003    1.000000e+00
000004    2.613237e-10
000005    2.295281e-10
000006    2.831511e-10
dtype: float64


In [6]:
risk_model.cov

,000001,000002,000003,000004,000005,000006,000007,000008
000001,0.097219,0.005132,0.012718,-0.011042,-0.021040,-0.002258,0.012836,-0.029536
000002,0.005132,0.112829,0.010618,0.011023,0.017567,-0.013539,-0.023365,0.013763
000003,0.012718,0.010618,0.115532,-0.009497,0.004587,0.011560,0.029916,0.002939
000004,-0.011042,0.011023,-0.009497,0.105560,0.003069,-0.024043,0.000922,0.025892
000005,-0.021040,0.017567,0.004587,0.003069,0.113839,-0.019018,0.003712,0.024478
000006,-0.002258,-0.013539,0.011560,-0.024043,-0.019018,0.099162,0.029616,-0.015033
000007,0.012836,-0.023365,0.029916,0.000922,0.003712,0.029616,0.125954,0.003735
000008,-0.029536,0.013763,0.002939,0.025892,0.024478,-0.015033,0.003735,0.141301


In [7]:
obj4 = TargetWeights(pd.Series(rets, index=stocks))

In [14]:
calculate_optimal_portfolio(obj4, [], None)

000001    0.0644
000002    0.0806
000003    0.1205
000004    0.0426
000005    0.0329
000006    0.0480
000007    0.0884
000008    0.0665
dtype: float64

In [15]:
rets

array([0.0644, 0.0806, 0.1205, 0.0426, 0.0329, 0.048 , 0.0884, 0.0665])